In [1]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
print(torch.cuda.is_available())

/scratch/AzureNfsServer_INPUT1/ads-nfs/zaohuixu/conda/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


In [2]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-70b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-70b-chat-hf",device_map="auto")

Loading checkpoint shards: 100%|██████████| 15/15 [00:28<00:00,  1.88s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 1 has a total capacty of 31.75 GiB of which 6.94 MiB is free. Process 170740 has 29.87 GiB memory in use. Process 349402 has 1.87 GiB memory in use. Of the allocated memory 1.02 GiB is allocated by PyTorch, and 1.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
p_qag="""
You are a Query/Answer generator that generates new 4 Queries or Answers which is based on old Queries and known spo informations. if the Answer is not clear you need to reply [],if the Answer is clear you need to generate Answers. The new Queries/Answer must be based on old Queries and spo informations. Now give the old Queries and spo informations:
{}
{}
"""

p_kve="""
You are a Subject-Predicate-Object extractor.I will give you a Doc you will extract Subject-Predicate-Object from the Doc. If no information is available then reply []
{}
Doc:{}
SPO:
"""

p_judge="""
You are a Query and answer evaluator, and you will use one Query and two answers to determine whether their conclusions are consistent.
Query:{}
answer1:{}
answer2:{}
"""

device='cuda'
# model.to(device)

def spo_extract(doc):
    example="""
Doc:One Law for the Woman is a 1924 American silent western film directed by Dell Henderson and starring Cullen Landis, Mildred Harris and Cecil Spooner.
SPO:
[
{"Subject":"One Law for the Woman","Predicate":"director","Object":"Dell Henderson"},
{"Subject":"One Law for the Woman","Predicate":"starring role","Object":"Cullen Landis"},
{"Subject":"One Law for the Woman","Predicate":"starring role","Object":"Mildred Harris"},
{"Subject":"One Law for the Woman","Predicate":"starring role","Object":"Cecil Spooner"},
{"Subject":"One Law for the Woman","Predicate":"premiere date","Object":"1924"},
{"Subject":"One Law for the Woman","Predicate":"country","Object":"American"},
{"Subject":"One Law for the Woman","Predicate":"theme","Object":"silent western film"},
]
"""
    input_text=p_kve.format(example,doc)
    inputs=tokenizer(input_text, return_tensors="pt")
    out = model.generate(
        input_ids=inputs["input_ids"]
    )
    out_text = tokenizer.decode(out[0])
    print(out_text)

def qa_generate(query,spo):
    example="""
old Query:
[
"Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?",
"Is the director of the movie \"Move\" (1970) the same nationality as the director of the film \"Méditerranée\" (1963)?",
"Do the director of \"Move\" (1970) and the director of \"Méditerranée\" (1963) hail from the same country?",
"Can it be confirmed whether the director of the film \"Move\" (1970) and the director of \"Méditerranée\" (1963) are of the same origin?"
]
spo informations:
[
{"Subject":"Move","Predicate":"director","Object":"Stuart Rosenberg"},
{"Subject":"Méditerranée","Predicate":"director","Object":"Jean-Daniel Pollet"},
]
new Querys:
[
"Are Stuart Rosenberg, the director of \"Move\" (1970), and Jean-Daniel Pollet, the director of \"Méditerranée\" (1963), from the same country?",
"Is Stuart Rosenberg, the director of \"Move\" (1970), and Jean-Daniel Pollet, the director of \"Méditerranée\" (1963), of the same nationality?",
"Do Stuart Rosenberg, the director of \"Move\" (1970), and Jean-Daniel Pollet, the director of \"Méditerranée\" (1963), share the same country of origin?",
"Is the country of origin the same for Stuart Rosenberg, director of \"Move\" (1970), and Jean-Daniel Pollet, director of \"Méditerranée\" (1963)?"
]
Answer:
[]
"""

    tmp="""
old Query:
{}
spo informations:
{}
"""
    input_text=p_kve.format(example,tmp.format(query,spo))
    inputs=tokenizer(input_text, return_tensors="pt")
    out = model.generate(
        input_ids=inputs["input_ids"]
    )
    out_text = tokenizer.decode(out[0])
    print(out_text)

# def evaluator(query,ans,ground_truth):
#     input_text=p_kve.format(query,ans,ground_truth)
#     inputs=tokenizer(input_text, return_tensors="pt")
#     out = model.generate(
#         input_ids=inputs["input_ids"].to(device)
#     )
#     out_text = tokenizer.decode(out[0])
#     print(out_text)


In [ ]:
doc="Hanro Smitsman, born in 1967 in Breda( Netherlands), is a writer and director of film and television."

query="""[
"Which country is Dell Henderson, the director of the film "One Law for the Woman" (1924), from?",
"Is Dell Henderson, the director of the film "One Law for the Woman" (1924), of American nationality?",
"From which country hails Dell Henderson, the director of "One Law for the Woman" (1924)?",
"Is the director of "One Law for the Woman" (1924), Dell Henderson, American by nationality?"
]"""
spo="""[
{"Subject":"George Delbert "Dell" Henderson","Predicate":"date of birth","Object":"July 5, 1877"},
{"Subject":"George Delbert "Dell" Henderson","Predicate":"date of death","Object":"December 2, 1956"},
{"Subject":"George Delbert "Dell" Henderson","Predicate":"nationality","Object":"Canadian-American"},
{"Subject":"George Delbert "Dell" Henderson","Predicate":"profession","Object":"actor, director, writer"}
]
"""

# spo_extract(doc)

qa_generate(query,spo)

In [ ]:
qa_generate("","")